Import Classes and Define Variables

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import scipy as sp
import pymongo
import pandas as pd
import nsepy
from datetime import datetime
from datetime import timedelta
from datetime import date
from pymongo import MongoClient
from nsepy import get_history
client=MongoClient("mongodb://localhost:27018/",username="stockadm",password="1qaz@WSX")
db=client["stockWarehouse"]
equityList=db["stocklist"]
stockWarehouse=db["stockWarehouse"]
equityListPd = pd.DataFrame(equityList.find({}))
equityListPd.shape[0]


8426

Data Collector Function

In [2]:
def dataCollector():
    equityListPd = pd.DataFrame(equityList.find({'Security Name':'Active','State':"A"}))
    collectionTime=2000
    basicSample=30
    cnt=0
    my_time = datetime.now()

    for rows in equityListPd['Issuer Name'].iteritems():
        print(cnt,rows[1],end="-")
        cnt= cnt + 1
        if stockWarehouse.find({'Symbol':rows[1]}).count() > 1 :
            for j in stockWarehouse.find({'Symbol':rows[1]},{'_id':0,'Date':1}).sort('Date',-1).limit(1) :
                collectionTime=((my_time - j['Date']).days - 1)
                fromDate=j['Date']
        iters=int(collectionTime/basicSample)
        if (collectionTime != 0):
            if collectionTime < 2000:
                basicSample=collectionTime
                iters=1
            dt=datetime.now()
            print("Iterations:",iters,end="|")
            for i in range(0,iters):

                startDate=dt - timedelta(days=basicSample)
                if( startDate <= fromDate):
                    startDate = (fromDate + timedelta(days=1))
                endDate=dt 
                print(i,"Collecting into between",startDate.date(),"-", endDate.date(),"@",datetime.now(),end="|")
                df=pd.DataFrame(get_history(symbol=rows[1] , start= startDate,end=endDate))
                df.reset_index(inplace=True)
                df['Date']=pd.to_datetime(df['Date'])
                dfLength=(len(df.index))
                if  dfLength > 0 :
                    data_dict=df.to_dict("records")
                    stockWarehouse.insert_many(data_dict)
                    tmpdf=pd.DataFrame(stockWarehouse.find({"Symbol":rows[i],"Date":{"$lte":endDate,"$gte":startDate}}))
                    print(df.index.size,tmpdf.index.size,end="|")

                else :
                    print("Marking as Inactive")
                    equityList.update_many({'Issuer Name':rows[1]},{'$set': {'Security Name':'Inactive'}})
                    break;
                dt=(startDate - timedelta(days=1))


In [3]:
def markInactiveStocks():
    reference=int(stockWarehouse.find({'Symbol':'SBIN'}).count()) - 100
    print(reference)
    tmp_count=0
    for i in equityList.find({},{'Issuer Name':1}):
        #print(i['Issuer Name'])
        tmp_count=(int(stockWarehouse.find({'Symbol':i['Issuer Name']}).count()))
        if tmp_count >= reference:
            print(i['Issuer Name'],end='|')
            equityList.update_many({'Issuer Name':i['Issuer Name']},{'$set': {'State':"A"}})

In [4]:
def markTrend_Crossover(stockdf,datadf,symbol):
    
    for i in range(0,stockdf.shape[0],1):
            if stockdf['Issuer Name'].iloc[i]==symbol:
                break;
    datadf['ewmdiff']=datadf['Close'].ewm(span=10,adjust=False).mean() - datadf['Close'].ewm(span=25,adjust=False).mean()
    nrows=datadf.shape[0]-1
    if datadf.iloc[nrows,15] > 0:
        stockdf.iloc[i,1]=1
    else:
        stockdf.iloc[i,1]=-1
    for j in range(datadf.shape[0]-1,2,-1):
        if datadf.iloc[j,15]*datadf.iloc[j-1,15] <= 0:
            stockdf.iloc[i,2]=datadf.shape[0]-j            
            break;
    if datadf.iloc[nrows,15] > datadf.iloc[nrows-1,15] and datadf.iloc[nrows-1,15] > datadf.iloc[nrows-2,15]:
        stockdf.iloc[i,3]='Expanding'
    else:
        stockdf.iloc[i,3]='Contracting'
    return(stockdf)

In [5]:
def singlePatternCandle(stockdf,datadf,symbol):
    for i in range(0,stockdf.shape[0],1):
        if stockdf['Issuer Name'].iloc[i]==symbol:
            break;
                
    pattern=''
    umbrellaFound=False
    if 'singleCandlePattern' in stockdf.columns:
        if pd.isnull(stockdf['singleCandlePattern'].iloc[i]):
            pattern=''
        else:
            pattern=stockdf['singleCandlePattern'].iloc[i]
    nrows=datadf.shape[0]-1
    Close=datadf['Close'].iloc[nrows]
    Open=datadf['Open'].iloc[nrows]
    High=datadf['High'].iloc[nrows]
    Low=datadf['Low'].iloc[nrows]
    if  Open > Close :
        candle=-1
    if  Open < Close :
        candle=1
    if  Open == Close :
        candle=0
####### Check MaruBozo
    
    if candle > 0:
        if ((High-Close)/(Close-Open)) < .05 and ((Open-Low)/(Close-Open))< .05:
            pattern=pattern + 'Bull Marubozo|' +'SL:' + str(Low)
    if candle < 0:
        if ((High-Open)/(Open-Close)) < .05 and ((Close-Low)/(Open-Close)) < .05:
            pattern=pattern + 'Bear Marubozo|' +'SL:' + str(High)
        
####### Check Paper Umbrella
    if candle > 0:
        if ((Open-Low)/(Close-Open)) > 2 and (High-Close)/(Close-Open) < .05:
            umbrellaFound=True
    if candle < 0:
        if ((Close-Low)/(Open-Close)) > 2 and (High-Open)/(Open-Close) < .05:
            umbrellaFound=True
    if umbrellaFound:
        if stockdf['trend'].iloc[i] == 1:
            pattern=pattern + 'Hanging Man ' +'SL:' + str(High)
        else:
            pattern=pattern + 'Hammer ' + 'SL:' + str(Low)
        
    stockdf['singleCandlePattern'].iloc[i]=pattern
    return(stockdf)
                
        
        

In [6]:
def rsiindicator(stockdf,datadf,symbol,term):
    for i in range(0,stockdf.shape[0],1):
        if stockdf['Issuer Name'].iloc[i]==symbol:
            break;
    nrows=datadf.shape[0]-1
    datadf['Diff']=datadf['Close'].diff()
    datadf['AVGU']=0
    datadf['AVGD']=0
    datadf.loc[datadf['Diff'] > 0,'AVGU' ] = datadf['Diff']
    datadf.loc[datadf['Diff'] < 0,'AVGD' ] = -datadf['Diff']
    AVGUT=datadf.iloc[nrows-1,17]/term + (term - 1)*datadf.iloc[(nrows-(term+1)):(nrows-2),17].mean()/term    
    AVGDT=datadf.iloc[nrows-1,18]/term + (term - 1)*datadf.iloc[(nrows-(term+1)):(nrows-2),18].mean()/term
    RS=AVGUT/AVGDT
    RSI=100-(100/(1+RS))
    stockdf.iloc[i,5]=RSI
    return(stockdf)

In [7]:
def getAllDaysOfWeek(year,day):  # Get days of year 
    d = datetime(2020,1,1,0,0,0)                    # January 1st
    d += timedelta(days = day - d.weekday())  # First Sunday
    while d.year == year and d < datetime.now():
        yield d
        d += timedelta(days = 7)


In [8]:
#dataCollector()

In [9]:
#markInactiveStocks()

In [11]:
equityList=db["stocklist"]
stockWarehouse=db["stockWarehouse"]

equityListdf=pd.DataFrame(equityList.find({'Security Name':'Active','State':"A"},{'Issuer Name':1,'_id':0}))
equityListdf['trend']=0
equityListdf['daysSinceCutOver']=0
equityListdf['trendnature']=''
equityListdf['singleCandlePattern']=''
equityListdf['RSI']=0
equityListdf['Close']=0
equityListdf['Profit']=0
capital_needed=0
equityListdf['invested_amount']=0    
equityListdf['Profit']=0
equityListdf['Capital']=0
equityListdf['trades']=0
cnt=0  
i=0
#print(equityListdf.columns)
for symbol in equityListdf['Issuer Name']:
    cnt=cnt+1
#    print(symbol,cnt)
    moneyinvested=0
    sharesbought=0
    profit=0  
    capital_needed=0
    trades=0
    for dt in stockWarehouse.find({'Symbol':symbol,'Date':{'$gte':datetime(2020,5,26)},'Close':{'$lte':2000},'Close':{'$gte':100}},{'_id':0,'Date':1}).sort('Date'):
        for dt1 in stockWarehouse.find({'Symbol':'SBIN','Date':{'$lte':dt['Date']}},{'_id':0,'Date':1}).sort('Date',-1).limit(1):
            refdate=dt1 # Set Reference Data as last date when SBI Stock Data was update
        for maxdate in stockWarehouse.find({'Symbol':symbol,'Date':{'$lte':dt['Date']}},{'_id':0,'Date':1}).sort('Date',-1).limit(1):
            if maxdate == refdate :  # Check if database has latest sttock info by comparing aginst SBI
                datadf=pd.DataFrame(stockWarehouse.find({'Symbol':symbol,'Date':{'$lte':dt['Date']}}).sort('Date',-1).limit(300))# Get top 300 row of stock in iteration
                datadf.set_index('Date',inplace=True)
                datadf.sort_index(inplace=True)
                equityListdf=markTrend_Crossover(equityListdf,datadf,symbol)
                equityListdf=rsiindicator(equityListdf,datadf,symbol,15)
                equityListdf.loc[equityListdf['Issuer Name'] == symbol,'Close']=datadf.iloc[-1,7]
                rsi=0
                trend=0
                tempnp=equityListdf.loc[equityListdf['Issuer Name'] == symbol].to_numpy()
                
                if tempnp[0,5] < 50 and tempnp[0,1] == 1 and tempnp[0,3] == 'Expanding' and tempnp[0,6] < 2000 :
                    #print("buy",maxdate['Date'].date(),end='|')
                    sharesbought=sharesbought+1
                    moneyinvested=moneyinvested + tempnp[0,6]
                    capital_needed=capital_needed+tempnp[0,6]
                else:
                    if (tempnp[0,5] > 80 or tempnp[0,1] == -1)  and sharesbought > 0:
                        #print("sell",tempnp[0,6],sharesbought,moneyinvested,maxdate['Date'].date())
                        profit=profit + ((tempnp[0,6] * sharesbought) - moneyinvested)
                        #capital_needed=capital_needed-(tempnp[0,6] * sharesbought)
                        sharesbought=0
                        trades=trades+1
                        moneyinvested=0
    if moneyinvested > 0:
        equityListdf.loc[equityListdf['Issuer Name'] == symbol,'invested_amount']=moneyinvested    
    equityListdf.loc[equityListdf['Issuer Name'] == symbol,'Profit']=profit
    equityListdf.loc[equityListdf['Issuer Name'] == symbol,'Capital']=capital_needed
    equityListdf.loc[equityListdf['Issuer Name'] == symbol,'trades']=trades
    print('Symbol',symbol,'Profit:',profit,'cnt:',cnt)

Symbol ABB Profit: -48.10000000000002
Symbol AMARAJABAT Profit: 0
Symbol HDFC Profit: 84.20000000000005
Symbol ASIANHOTNR Profit: 0
Symbol ATUL Profit: 0
Symbol AUTOLITIND Profit: 0
Symbol BAJAJHIND Profit: 0
Symbol BAJFINANCE Profit: 0
Symbol BANCOINDIA Profit: 0
Symbol CENTURYTEX Profit: 0
Symbol BATAINDIA Profit: 0
Symbol BANARISUG Profit: 0
Symbol BEL Profit: 0
Symbol BEML Profit: 0
Symbol BEPL Profit: 0
Symbol CENTEXT Profit: 0
Symbol CESC Profit: -83.70000000000027
Symbol CHAMBLFERT Profit: 19.350000000000023
Symbol EXIDEIND Profit: -4.400000000000006
Symbol CIPLA Profit: 0
Symbol CRISIL Profit: 240.45000000000005
Symbol DABUR Profit: 37.90000000000009
Symbol DALMIASUG Profit: 0
Symbol ARVIND Profit: 0
Symbol BHEL Profit: 0
Symbol HINDPETRO Profit: 0
Symbol IFCI Profit: 0
Symbol MTNL Profit: 0
Symbol MRPL Profit: 0
Symbol SAIL Profit: 0
Symbol TITAN Profit: 0
Symbol IDBI Profit: 0
Symbol DCW Profit: 0
Symbol DHAMPURSUG Profit: 0
Symbol DRREDDY Profit: 0
Symbol EIDPARRY Profit: 0


Symbol MOTHERSUMI Profit: 0
Symbol HAVELLS Profit: 0
Symbol IGARASHI Profit: 65.15
Symbol SYMPHONY Profit: -58.69999999999982
Symbol RSSOFTWARE Profit: 0
Symbol TTKPRESTIG Profit: 0
Symbol SURANAT&P Profit: 0
Symbol ONWARDTEC Profit: 0
Symbol CENTUM Profit: 0
Symbol KEI Profit: 0
Symbol GSCLCEMENT Profit: 0
Symbol AVTNPL Profit: 0
Symbol NAHARINDUS Profit: 0
Symbol VADILALIND Profit: -51.850000000000136
Symbol ADFFOODS Profit: 0
Symbol HERITGFOOD Profit: 15.500000000000028
Symbol CCL Profit: 0
Symbol RICOAUTO Profit: 0
Symbol MUNJALSHOW Profit: 0
Symbol JAMNAAUTO Profit: 0
Symbol SUNCLAYLTD Profit: 0
Symbol MUNJALAU Profit: 0
Symbol JAYBARMARU Profit: 85.39999999999998
Symbol SICAL Profit: 0
Symbol RATNAMANI Profit: 0
Symbol VESUVIUS Profit: 0
Symbol MARALOVER Profit: 0
Symbol TRIDENT Profit: 0
Symbol NAGREEKEXP Profit: 0
Symbol ZODIACLOTH Profit: 0
Symbol KITEX Profit: 0
Symbol SHANTIGEAR Profit: 0
Symbol HONDAPOWER Profit: 133.5999999999999
Symbol HITECHGEAR Profit: 0
Symbol ELGIEQUI

Symbol FIEMIND Profit: 168.5999999999999
Symbol DCBBANK Profit: 0
Symbol NAUKRI Profit: 0
Symbol LUMAXTECH Profit: 0
Symbol NETWORK18 Profit: 0
Symbol TV18BRDCST Profit: 0
Symbol TIIL Profit: 0
Symbol REDINGTON Profit: 0
Symbol CINELINE Profit: 0
Symbol FSL Profit: 0
Symbol PGIL Profit: 0
Symbol AHLUCONT Profit: 0
Symbol PFC Profit: 0
Symbol INDIANB Profit: 0
Symbol SMSPHARMA Profit: 0
Symbol MINDTREE Profit: 104.75
Symbol IDEA Profit: 0
Symbol PAGEIND Profit: 0
Symbol AMDIND Profit: 0
Symbol ASTRAL Profit: 60.85000000000002
Symbol IBREALEST Profit: 0
Symbol CAMLINFINE Profit: 0
Symbol ICRA Profit: 0
Symbol DISHTV Profit: 0
Symbol SRHHYPOLTD Profit: 65.0
Symbol FORTIS Profit: 0
Symbol DELTACORP Profit: 0
Symbol INSECTICID Profit: 363.5999999999998
Symbol MCDHOLDING Profit: 0
Symbol ASAHISONG Profit: 46.89999999999998
Symbol TIMETECHNO Profit: 0
Symbol HGS Profit: 21.799999999999955
Symbol NELCAST Profit: 0
Symbol MEGH Profit: 0
Symbol DLF Profit: -13.549999999999983
Symbol TARMAT Profi

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered 

Symbol OMAXE Profit: 0
Symbol CENTRALBK Profit: 0
Symbol KPRMILL Profit: 0
Symbol ASIANTILES Profit: 0
Symbol TAKE Profit: 0
Symbol PURVA Profit: 0
Symbol MOTILALOFS Profit: 0
Symbol POWERGRID Profit: 5.5
Symbol KSCL Profit: 0
Symbol MAANALU Profit: 0
Symbol RELIGARE Profit: 0
Symbol ADANIPORTS Profit: 0
Symbol EDELWEISS Profit: 0
Symbol KOLTEPATIL Profit: -16.950000000000017
Symbol JYOTHYLAB Profit: 0
Symbol ECLERX Profit: 66.65000000000003
Symbol TRIL Profit: 0
Symbol BRIGADE Profit: 0
Symbol PPAP Profit: -5.900000000000006
Symbol ARIES Profit: 0
Symbol RPOWER Profit: 0
Symbol JKIL Profit: 0
Symbol CORDSCABLE Profit: 0
Symbol KNRCON Profit: 0
Symbol ONMOBILE Profit: 0
Symbol SHRIRAMEPC Profit: 0
Symbol GSS Profit: 0
Symbol NAHARCAP Profit: 0
Symbol VGUARD Profit: 0
Symbol RECLTD Profit: 18.30000000000001
Symbol TWL Profit: 0
Symbol KIRIINDUS Profit: 0
Symbol BAJAJFINSV Profit: 0
Symbol BAJAJ-AUTO Profit: 0
Symbol RPGLIFE Profit: 0
Symbol RBL Profit: 14.099999999999966
Symbol RANEENGI

In [12]:
#print(equityListdf.loc[equityListdf['Issuer Name'] == symbol,'RSI'][1])
#print(equityListdf.loc[equityListdf['Issuer Name'] == symbol,'RSI'][0])
#print(equityListdf.loc[equityListdf['Issuer Name'] == symbol,'RSI'][0])
equityListdf.to_csv("C:\\Users\\khanna's\\Documents\\Stock Trend Analysis\\result.csv")

In [13]:
stockWarehouse.find({'Symbol':'SBIN','Date':{'$lte': {'Date': datetime(2020, 7, 24, 0, 0)} }},{'_id':0,'Date':1}).sort('Date',-1).limit(1)
print(dt['Date'])

2020-07-24 00:00:00
